In [119]:
import pysam
import numpy as np
import pandas as pd
from tqdm import tqdm



In [120]:
Am2 = np.load("cap2_ann_Am.npy", allow_pickle = True)
Am1 = np.load("cap1_ann_Am.npy", allow_pickle = True)

m6Am2 = np.load("cap2_ann_m6Am.npy", allow_pickle = True)
m6Am1 = np.load("cap1_ann_m6Am.npy", allow_pickle = True)

Am  = np.load("amb_ann_Am.npy", allow_pickle = True)
m6Am  = np.load("amb_ann_m6Am.npy", allow_pickle = True)

print(Am2[0])

['chr17' '782279' '782280' '-' '1'
 'CTCTGCGAGCAGCCATGATTCCCCCGCACGCAGCCGTCAC']


In [121]:
from pyfaidx import Fasta
file_path = "/Volumes/Extreme SSD/HP/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna"
fasta = Fasta(file_path)

In [122]:
#return the base at specified chromosome and position 
def chr_loc(chromosome, position):
    try:
        basepair = fasta[chromosome][position - 1]  # pyfaidx uses 0-based indexing internally
        return basepair
    except KeyError:
        return f"Chromosome {chromosome} not found in the FASTA file."
    except IndexError:
        return f"Position {position} is out of range for chromosome {chromosome}."

In [124]:
def A_m6A_array(tsn_mod_array):
    m6A_array = []
    A_array = []
    for i in range(len(tsn_mod_array)):
        for j in range(len(tsn_mod_array[i][5])):
            if j>0 and tsn_mod_array[i][5][j] == 'A':
                m6A_array.append(tsn_mod_array[i][0])
                m6A_array.append(int(tsn_mod_array[i][1])+j+1)
                m6A_array.append(int(tsn_mod_array[i][2])+j+1)
                m6A_array.append(tsn_mod_array[i][3])
                m6A_array.append(tsn_mod_array[i][4])
            if j>0 and tsn_mod_array[i][5][j] == 'G' and chr_loc(str(tsn_mod_array[i][0]), int(tsn_mod_array[i][1])) == 'A':
                A_array.append(tsn_mod_array[i][0])
                A_array.append(int(tsn_mod_array[i][1])+j+1)
                A_array.append(int(tsn_mod_array[i][2])+j+1)
                A_array.append(tsn_mod_array[i][3])
                A_array.append(j+1)  
    m6A_array = np.array(m6A_array)
    m6A_array = m6A_array.reshape((int(m6A_array.size/5), 5))
    
    A_array = np.array(A_array)
    A_array = A_array.reshape((int(A_array.size/5), 5))
    return m6A_array, A_array

m6A_Am2, A_An2 = A_m6A_array(Am2)

m6A_m6Am2, A_m6Am2 = A_m6A_array(m6Am2) 

m6A_Am1, A_Am1 = A_m6A_array(Am1) 

m6A_m6Am1, A_m6Am1 = A_m6A_array(m6Am1) 


In [127]:
def annotate(mod_array):
    dict_sites = {}
    i = 0
    for elt in mod_array:
        key = (elt[0], elt[1], elt[2], elt[3])
        if key in dict_sites:
            dict_sites[key][1] += 1  # Increment the count if the key already in dictionary
        else:
            dict_sites[key] = [elt, 1] # otherwise the count is 1

    mod_sites = []
    mod_ann = [(k, v) for k, v in dict_sites.items()]
    for i in range(len(mod_ann)):
        for j in range(5):
            mod_sites.append(mod_ann[i][1][0][j])
        mod_sites.append(mod_ann[i][1][1])

    mod_sites = np.array(mod_sites)
    mod_sites = mod_sites.reshape((int(mod_sites.size/6), 6))
    return mod_sites
m6A_Am1_sites = annotate(m6A_Am1)
A_Am1_sites = annotate(A_Am1)
m6A_m6Am1_sites = annotate(m6A_m6Am1)
A_m6Am1_sites = annotate(A_m6Am1)
m6A_Am2_sites = annotate(m6A_Am2)
A_Am1_sites = annotate(A_Am2)
m6A_m6Am2_sites = annotate(m6A_m6Am2)
A_m6Am2_sites = annotate(A_m6Am2)



In [116]:
np.save("A_m6Am1_ann.npy", A_m6Am1_sites)
np.save("m6A_m6Am1_ann.npy", m6A_m6Am1_sites)
np.save("A_Am1_ann.npy", A_Am1_sites)
np.save("m6A_Am1_ann.npy", m6A_Am1_sites)

np.save("A_m6Am2_ann.npy", A_m6Am2_sites)
np.save("m6A_m6Am2_ann.npy", m6A_m6Am2_sites)
np.save("A_Am2_ann.npy", A_Am2_sites)
np.save("m6A_Am2_ann.npy", m6A_Am2_sites)


[['chr17' '782283' '782284' '-' '4' '1']
 ['chr17' '782285' '782286' '-' '6' '1']
 ['chr17' '782287' '782288' '-' '8' '1']
 ['chr17' '782290' '782291' '-' '11' '1']
 ['chr17' '782295' '782296' '-' '16' '1']
 ['chr17' '782302' '782303' '-' '23' '1']
 ['chr17' '782305' '782306' '-' '26' '1']
 ['chr17' '782309' '782310' '-' '30' '1']
 ['chr17' '782312' '782313' '-' '33' '1']
 ['chr17' '782315' '782316' '-' '36' '1']
 ['chr10' '988438' '988439' '+' '2' '1']
 ['chr10' '988443' '988444' '+' '7' '1']
 ['chr10' '988444' '988445' '+' '8' '1']
 ['chr10' '988445' '988446' '+' '9' '1']
 ['chr10' '988447' '988448' '+' '11' '1']
 ['chr10' '988449' '988450' '+' '13' '1']
 ['chr10' '988454' '988455' '+' '18' '1']
 ['chr10' '988460' '988461' '+' '24' '1']
 ['chr10' '988462' '988463' '+' '26' '1']
 ['chr10' '988464' '988465' '+' '28' '1']]
[['chr16' '53627' '53628' '-' '3'
  'TCTCGAACTCCGCAGGCTCCAACTCCCGGCAGCTCCCACT']
 ['chr11' '236988' '236989' '+' '2'
  'AGCCATCCCCGCGGTGCTGACATCCCGGTTGTTCTTCTGT']
 ['c